In [1]:
#I will be using this notebook to build my final Capstone project
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
#PART 1 -> Scraping the WIKI page and putting the findings into a dataframe.

from urllib.request import urlopen
import urllib.request
from bs4 import BeautifulSoup
import re

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

#print(soup.prettify())
all_tables=soup.find_all("table")
all_tables

right_table = soup.find('table',class_='wikitable sortable')

PostalCode=[]
Borough=[]
Neighborhood=[]

for row in right_table.find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        PostalCode.append(cells[0].text.strip())
        Borough.append(cells[1].text.strip())
        Neighborhood.append(cells[2].text.strip())

df=pd.DataFrame(PostalCode,columns=['PostalCode'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood

#df['PostalCode'] = df['PostalCode'].astype('str')
#df['Borough'] = df['Borough'].astype('str')
#df['Neighborhood'] = df['Neighborhood'].astype('str')
df=df.astype('str')
df=df[df.Borough != 'Not assigned']
if df.Neighborhood is 'Not assigned':
    df.Neighborhood = Borough

group_df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join)
new_df=pd.DataFrame(group_df)
new_df2=new_df.reset_index()
new_df2['Neighborhood'] = new_df2['Neighborhood'].str.replace(' /',',')
new_df2.shape
new_df2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [3]:
#PART 2 -> Find the long and lat of each neighborhood.

!pip install geocoder 
import geocoder # import geocoder

def get_geocoder(postal_code_from_df):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df))
        lat_lng_coords=g.latlng
    neigh_lat = lat_lng_coords[0]
    neigh_lng = lat_lng_coords[1]
    return neigh_lat, neigh_lng

update_df=new_df2
update_df['Latitude'], update_df['Longitude'] = zip(*update_df['PostalCode'].apply(get_geocoder))

update_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [4]:
#PART 3 -> Explore the findings (through the next few cells)
!pip install geopy
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [5]:
#PART 3
!pip install folium
import folium # plotting library

In [6]:
#PART 3

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(update_df['Latitude'], update_df['Longitude'], update_df['Borough'], update_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
#import pandas as pd 
#import requests 
#from bs4 import BeautifulSoup 

#req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
#soup = BeautifulSoup(req.content,'lxml') 
#table = soup.find_all('table')[0] 
#df = pd.read_html(str(table)) 

#neighborhood=pd.DataFrame(df[0])
#neighborhood

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
#PART 3
import requests
print("done")

done


In [10]:
#PART 3
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng, in zip (names, latitudes, longitudes):
        print(name)
    
        #API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
        #GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
    
        #Return relevant info for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    
    return(nearby_venues)

In [11]:
#PART 3
toronto_venues = getNearbyVenues(names=update_df['Neighborhood'],
                                   latitudes=update_df['Latitude'],
                                   longitudes=update_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [12]:
#PART 3
print(toronto_venues.shape)
toronto_venues.head()

(2276, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
2,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
3,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Swiss Chalet,43.768122,-79.190493,Restaurant
4,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Shoppers Drug Mart,43.770282,-79.185012,Pharmacy


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,68,68,68,68,68,68
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",45,45,45,45,45,45
Business reply mail Processing CentrE,100,100,100,100,100,100


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 uniques categories.


In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

first_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0,'Neighborhood',first_col)

toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_onehot.shape

(2276, 264)

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.0000,0.00,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.014706,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.014706
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.022222,0.000000,0.000000,0.022222,0.000000,0.022222,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000
8,Business reply mail Processing CentrE,0.0,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.0100,0.00,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000


In [18]:
toronto_grouped.shape


(93, 264)

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0    Breakfast Spot  0.25
1   Badminton Court  0.25
2       Supermarket  0.25
3  Sushi Restaurant  0.25
4               ATM  0.00


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store   0.1
1        Coffee Shop   0.1
2     Sandwich Place   0.1
3       Dance Studio   0.1
4        Pizza Place   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2  Ice Cream Shop  0.05
3   Deli / Bodega  0.05
4        Pharmacy  0.05


----Bayview Village----
                        venue  freq
0  Construction & Landscaping   0.5
1                       Trail   0.5
2                 Music Store   0.0
3                Noodle House   0.0
4                   Nightclub   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Pizza Place  0.10
1         Coffee Shop  0.10
2  Italian Restaurant  0.10
3      Sandwich Place  0.10
4  

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Sushi Restaurant,Badminton Court,Supermarket,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,"Alderwood, Long Branch",Sandwich Place,Pharmacy,Convenience Store,Pizza Place,Gas Station,Pub,Athletics & Sports,Gym,Coffee Shop,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Gas Station,Restaurant,Supermarket,Fried Chicken Joint,Diner,Sandwich Place,Deli / Bodega
3,Bayview Village,Construction & Landscaping,Trail,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Pizza Place,Italian Restaurant,Sandwich Place,Restaurant,Pharmacy,Juice Bar,Greek Restaurant,Comfort Food Restaurant,Butcher


In [29]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [33]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = update_df
#toronto_merged['Cluster Labels']=kmeans.labels

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,1.0,Trail,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Dog Run
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,2.0,Pizza Place,Fast Food Restaurant,Breakfast Spot,Beer Store,Restaurant,Supermarket,Bank,Fried Chicken Joint,Coffee Shop,Intersection
3,M1G,Scarborough,Woburn,43.771545,-79.218135,2.0,Coffee Shop,Insurance Office,Korean Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,1.0,Construction & Landscaping,Trail,Gaming Cafe,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [34]:
drop_toronto_merged = toronto_merged.dropna(how='any', subset=['Cluster Labels'])
toronto_merged_final=drop_toronto_merged.reset_index()
toronto_merged_final.head()


,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,1.0,Trail,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Dog Run
1,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,2.0,Pizza Place,Fast Food Restaurant,Breakfast Spot,Beer Store,Restaurant,Supermarket,Bank,Fried Chicken Joint,Coffee Shop,Intersection
2,3,M1G,Scarborough,Woburn,43.771545,-79.218135,2.0,Coffee Shop,Insurance Office,Korean Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Donut Shop
3,4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,1.0,Construction & Landscaping,Trail,Gaming Cafe,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725,2.0,Sandwich Place,Fast Food Restaurant,Indian Restaurant,Big Box Store,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant


In [35]:
!pip install folium
import folium # plotting library

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_final['Latitude'], toronto_merged_final['Longitude'], toronto_merged_final['Neighborhood'], toronto_merged_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0.0,Bakery,Wine Shop,Construction & Landscaping,Brewery,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
11,Scarborough,0.0,Auto Garage,Yoga Studio,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Dog Run
17,North York,0.0,Residential Building (Apartment / Condo),Furniture / Home Store,Dog Run,Park,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
25,North York,0.0,Food & Drink Shop,Park,Bus Stop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Donut Shop
37,East Toronto,0.0,Pub,Park,Health Food Store,Church,Trail,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
41,East Toronto,0.0,Park,Business Service,Discount Store,Grocery Store,Bus Line,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Yoga Studio
43,East Toronto,0.0,Night Market,Baseball Field,Government Building,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Yoga Studio,Dumpling Restaurant
44,Central Toronto,0.0,Bus Line,Swim School,Bus Stop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,Food Court
45,Central Toronto,0.0,Breakfast Spot,Park,Gym,Department Store,Food & Drink Shop,Cuban Restaurant,Cupcake Shop,Food,Flower Shop,Fish Market
46,Central Toronto,0.0,Playground,Garden,Park,Gym Pool,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant
